In [ ]:
import asana  
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from asana import ApiClient, TasksApi


# Take info from Asana

In [104]:
import asana
print(dir(asana)) # only for see and check

['AccessRequestsApi', 'AllocationsApi', 'ApiClient', 'AttachmentsApi', 'AuditLogAPIApi', 'BatchAPIApi', 'Configuration', 'CustomFieldSettingsApi', 'CustomFieldsApi', 'CustomTypesApi', 'EventsApi', 'ExportsApi', 'GoalRelationshipsApi', 'GoalsApi', 'JobsApi', 'MembershipsApi', 'OrganizationExportsApi', 'PortfolioMembershipsApi', 'PortfoliosApi', 'ProjectBriefsApi', 'ProjectMembershipsApi', 'ProjectStatusesApi', 'ProjectTemplatesApi', 'ProjectsApi', 'ReactionsApi', 'RulesApi', 'SectionsApi', 'StatusUpdatesApi', 'StoriesApi', 'TagsApi', 'TaskTemplatesApi', 'TasksApi', 'TeamMembershipsApi', 'TeamsApi', 'TimePeriodsApi', 'TimeTrackingEntriesApi', 'TypeaheadApi', 'UserTaskListsApi', 'UsersApi', 'WebhooksApi', 'WorkspaceMembershipsApi', 'WorkspacesApi', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'absolute_import', 'api', 'api_client', 'configuration', 'pagination', 'rest']


In [105]:
ASANA_TOKEN = "DONT SHARE_YOUR TOKEN!! "
PROJECT_GID = "1211535446206848"


def fetch_asana_tasks():
    client = ApiClient()
    client.configuration.access_token = ASANA_TOKEN
    tasks_api = TasksApi(client)

    opts = {
        "opt_fields": "gid,name,assignee,status,notes,created_at,due_on"
    }

    tasks_generator = tasks_api.get_tasks_for_project(project_gid=PROJECT_GID, opts=opts)

    tasks = []
    for task in tasks_generator:  
        tasks.append({
            "id": task.get("gid"),
            "name": task.get("name"),
            "assignee": task.get("assignee", {}).get("name") if task.get("assignee") else None,
            "status": task.get("status"),
            "notes": task.get("notes"),
            "created_at": task.get("created_at"),
            "due_on": task.get("due_on")
        })
    return tasks


# convert documents

In [106]:
tasks = fetch_asana_tasks()
print(f"Fetched {len(tasks)} tasks")

documents = [
    Document(
        page_content=(t["name"] + "\n" + (t["notes"] or "")),
        metadata=t
    )
    for t in tasks
]

Fetched 4 tasks


In [107]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunked_docs = splitter.split_documents(documents)

print(f"Created {len(chunked_docs)} text chunks")

Created 4 text chunks
